In [1]:
IRT_DIMS = 1
TEST = True

In [2]:
import sys, os, shutil
import pickle
import time
import numpy as np
from utils import (mymode, load_data_heterogeneous, create_data_object_heterogeneous)

from IRT import MIRT_2PL
from Heterogeneous_embedder import EmbedderHeterogeneous, train_embedder_heterogeneous, test_embedder_heterogeneous
from manage_experiments import perform_cross_validation

import torch
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import pearsonr

MAXES = 10
MINEPOCHS = 5

In [3]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
# Initialise
parameters = {
    'df_name': None,
    'epochs': 10000,
    'learning_rate': 0.005,
    'weight_decay': 0,
    'early_stopping': 200,
    'n_splits': 10,
    'device': 'cuda:0',
    'batch_size': 1024
    }

if IRT_DIMS > 0:
    parameters['model_type'] = 'IRT'
    parameters['hidden_dims'] = IRT_DIMS
    parameters['lambda1'] = 0.01
    parameters['lambda2'] = 0.02
    OUTNAME = 'IRT'
else:
    parameters['model_type'] = 'GNN'
    parameters['hidden_dims'] = [16,8]
    OUTNAME = 'SAGE' 

In [5]:
if TEST: 
    #DATA_FILE = 'data/mindsteps_set_matrix_20000'    
    DATA_FILE = 'data/mindsteps_set_test_matrix'    
else:
    DATA_FILE = 'data/mindsteps_set_full' 

In [6]:
df = load_data_heterogeneous(DATA_FILE)
data, df_student, df_item, df_edge = create_data_object_heterogeneous(df, return_aux_data=True)

(327841, 24)


In [7]:
df.columns

Index(['studentId', 'code', 'motherTongue', 'Gender', 'scale', 'matrix',
       'IRT_difficulty', 'topic', 'responseformat', 'textlength', 'score',
       'age', 'grade', 'viewingTime', 'timestamp', 'useCase', 'assessmentId',
       'ability', 'frequency', 'previous_sessions', 'years_from_start',
       'matdiff', 'matcode', 'domain'],
      dtype='object')

In [8]:
df.head()

,studentId,code,motherTongue,Gender,scale,matrix,IRT_difficulty,topic,responseformat,textlength,...,timestamp,useCase,assessmentId,ability,frequency,previous_sessions,years_from_start,matdiff,matcode,domain
0,0,0,0.0,0.0,dles,D.2.B.1.f,0.268748,Zeitungsartikel,NaN,NaN,...,7.643058e-312,ms-kber,5E9436B294DBE2FB740ADDCF2B883ECB2254B5F0D37D8E...,2.667269,13.333162,2,0.15,,,d
1,0,1,0.0,0.0,dles,D.2.B.1.h,1.217388,Textsorten erkennen,single choice,NaN,...,7.643061e-312,ms-kber,5E9436B294DBE2FB740ADDCF2B883ECB2254B5F0D37D8E...,2.667269,13.333162,2,0.15,,,d
2,0,2,0.0,0.0,dles,D.2.B.1.h,1.584145,Wissenstexte: Mix,single choice,NaN,...,7.845624e-312,ms-kber,E109B3CDD31EDCFE7F48665587372C60A3FAEBB8571544...,2.421279,4.137915,6,1.45,,,d
3,0,3,0.0,0.0,dles,D.2.B.1.h,1.584145,Wissenstexte: Mix,single choice,NaN,...,7.851631e-312,ms-kber,319A0FE3AFDC0A30C4C1B26577B5B19FE66E918A181354...,2.174872,7.432378,11,1.48,,,d
4,0,4,0.0,0.0,dles,D.2.C.1.h,1.172410,Geschichten lang,single choice,NaN,...,7.643059e-312,ms-kber,5E9436B294DBE2FB740ADDCF2B883ECB2254B5F0D37D8E...,2.667269,13.333162,2,0.15,,,d


edge_dim = data['student', 'responds', 'item'].edge_attr.shape[1]
if IRT_DIMS > 0:
    model = MIRT_2PL(IRT_DIMS, edge_dim, data)
else:
    model = EmbedderHeterogeneous( 
        n_students =  data['student'].x.size(0),
        n_items = data['item'].x.size(0),
        student_inchannel = data['student'].x.size(1),
        item_inchannel = data['item'].x.size(1),
        hidden_channels=parameters['hidden_dims'],
        edge_channel=edge_dim,
        metadata=data.metadata()
        ).to(device)
    
model = model.to(device)

In [9]:
output_dict, model = perform_cross_validation(data, parameters, save_embeddings=True, save_subgraph=True, final_fit=True)

0it [00:00, ?it/s]
  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:00<00:00,  7.10it/s]

 50%|█████     | 2/4 [00:00<00:00,  7.01it/s]

tensor([[ 2.7158],
        [ 0.7591],
        [ 1.5490],
        ...,
        [-1.6551],
        [-1.6920],
        [-1.6109]], grad_fn=<AddBackward0>)
tensor([[-0.6175],
        [-0.0837],
        [-0.8989],
        ...,
        [ 1.1734],
        [-2.7174],
        [-3.6456]], grad_fn=<AddBackward0>)




 75%|███████▌  | 3/4 [00:00<00:00,  7.07it/s]

100%|██████████| 4/4 [00:00<00:00,  7.52it/s]

tensor([[ 0.4209],
        [ 2.0413],
        [-6.6112],
        ...,
        [-0.6056],
        [-1.9882],
        [-1.4344]], grad_fn=<AddBackward0>)
tensor([[-1.8504],
        [-1.0298],
        [ 1.8273],
        ...,
        [-0.5636],
        [-0.4838],
        [ 0.1397]], grad_fn=<AddBackward0>)
[0 0 0 ... 0 1 0]
[1 0 1 ... 1 0 0]
[1 0 1 ... 0 0 1]
[1 1 1 ... 1 1 0]

Epoch: 001, Loss: 31264.0137, Val: 0.4870, Test: 0.4957
[1 0 1 ... 0 0 1]
[1 1 1 ... 1 1 0]


[0 0 0 ... 0 1 0]
[1 0 1 ... 1 0 0]


...



 50%|█████     | 2/4 [00:00<00:00, 10.61it/s]

tensor([[ 0.3600],
        [ 0.2930],
        [ 0.3647],
        ...,
        [-0.2417],
        [-0.3385],
        [-0.0160]], grad_fn=<AddBackward0>)
tensor([[-0.0427],
        [-0.0567],
        [ 0.0418],
        ...,
        [ 1.0678],
        [ 1.0403],
        [ 1.0581]], grad_fn=<AddBackward0>)




  4%|▎         | 356/10000 [03:49<1:43:36,  1.55it/s]
0it [03:49, ?it/s]

tensor([[-1.5702],
        [-1.6002],
        [-1.5806],
        ...,
        [-1.4101],
        [-1.4704],
        [-1.4120]], grad_fn=<AddBackward0>)
[1 0 0 ... 1 1 0]
[1 0 1 ... 1 0 0]


KeyboardInterrupt: 

In [ ]:
output_dict.keys()
output_dict['AUC_0_test']

In [ ]:
unique_scales = df_item['scale'].unique()
unique_domains = df_item['domain'].unique()
unique_matdiff = df_item['matdiff'].sort_values().unique()
#scale_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_scales)])
#domain_colors = dict([(c, plt.cm.tab10(i)) for i, c in enumerate(unique_domains)])

In [ ]:
train_losses = output_dict['losses_0']
train_edge_indices, val_edge_indices, test_edge_indices = output_dict['indices_0']

train_data = output_dict['train_subgraph_data'] 
val_data = output_dict['val_subgraph_data'] 
test_data = output_dict['test_subgraph_data']

aux_data = (df, df_student, df_item, df_edge, 
    #clustering_indices, 
    train_losses, #test_losses, test_aucs, 
    train_edge_indices, val_edge_indices, test_edge_indices, 
    data, train_data, val_data, test_data,
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

with open(f'./results/{OUTNAME}_aux_data.pkl', 'wb') as handle:
    pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# SAVE EVERYTHING
torch.save(model, f'./results/{OUTNAME}.pth.tar')

In [ ]:
train_indices = np.arange(len(train_losses))
fig = plt.figure()
plt.plot(train_indices, train_losses, c='blue')
plt.ylabel('Loss')
plt.xlabel('Epoch')

fig.tight_layout()

In [ ]:
# save IRT parameters to matrix

if OUTNAME == 'IRT' and IRT_DIMS == 1:    
    #z_dict = model.get_embeddings(train_data.to(device))
    z_dict = output_dict['embedding_0']
    df_item['IRT1_difficulty'] = -z_dict['item']
    df_item['IRT1_discrimination'] = z_dict['offset']
    ability = z_dict['ability']
    df_edge['IRT1_ability'] = ability.ravel()
    
    aux_data = (df, df_student, df_item, df_edge, 
    train_losses, 
    train_edge_indices, val_edge_indices, test_edge_indices,
    data, train_data, val_data, test_data,
    unique_scales, unique_domains, unique_matdiff, 
    DATA_FILE)

    with open(f'./results/{OUTNAME}_aux_data_IRT1.pkl', 'wb') as handle:
        pickle.dump(aux_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    #fig = plt.figure()
    #plt.plot(df_item['IRT_difficulty'], df_item['IRT1_difficulty'], 'b.')
    df_item_clean = df_item.dropna(subset=['IRT_difficulty', 'IRT1_difficulty'])
    
    print(pearsonr(df_item_clean['IRT1_difficulty'], df_item_clean['IRT_difficulty']))
    
    edge_feat = train_data['student', 'responds', 'item'].edge_attr.detach().cpu().numpy()
    df_edge_clean = df_edge.dropna(subset=['IRT1_ability', 'ability', 'age'])
    fig = plt.figure()
    plt.plot(df_edge_clean['age'], df_edge_clean['IRT1_ability'], 'b.')
    print(pearsonr(df_edge_clean['age'], df_edge_clean['IRT1_ability']))

    fig = plt.figure()
    plt.plot(df_edge_clean['ability'], df_edge_clean['IRT1_ability'], 'b.')
    print(pearsonr(df_edge_clean['ability'], df_edge_clean['IRT1_ability']))
    